In [ ]:
import numpy as np
import pickle as pickle
import pandas as pd
import math
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time
# import pyeeg as pe
import pywt
import pickle
from itertools import cycle

from sklearn import svm
from sklearn.preprocessing import normalize
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

import tensorflow as tf
import keras
from tensorflow.keras.utils import to_categorical 
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import BatchNormalization, Activation, Dropout, Flatten, Dense
from keras.layers import Bidirectional, TimeDistributed
from keras.layers import LSTM, GRU, SimpleRNN
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.regularizers import l2

from tensorflow.keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

In [ ]:
def label_mapping(valence, arousal):
    # HVHA, HVLA, LVHA, and LVLA
    
    
    if (valence > 5 and arousal > 5):
        label = 1,1 #'HVHA'
    elif (valence > 5 and arousal <= 5):
        label = 1,0 #'HVLA'
    elif (valence <= 5 and arousal > 5):
        label = 0,1 #'LVHA'
    elif (valence <= 5 and arousal <= 5):
        label = 0,0 #'LVLA'
    
    return label

In [ ]:
def map_label(label):
    #print(label)
    new_label = label_mapping(label[0], label[1]) 
    return new_label

In [ ]:
def fft(X, Band, Fs):
  # PYEEG library's bin_power function
  C = np.fft.fft(X)
  C = abs(C)
  Power = np.zeros(len(Band) - 1)
  for Freq_Index in range(0, len(Band) - 1):
      Freq = float(Band[Freq_Index])
      Next_Freq = float(Band[Freq_Index + 1])
      Power[Freq_Index] = sum(
          C[int(np.floor(Freq / Fs * len(X))): 
              int(np.floor(Next_Freq / Fs * len(X)))]
      )
  Power_Ratio = Power / sum(Power)
  return Power, Power_Ratio

In [ ]:
channel = [1,2,3,4,5,6,7,8,9,10,11,12,13,14]
channel_test = [0, 16] # Fp1, Fp2

window_size = 512 #Averaging band power of 2 sec
step_size = 32 #256 #Each 1 sec update once 
subject_list=['01','02','03','04','05','06','07','09','10'
    ,'11','12','13','14','15','16','17','18','19','20'
    ,'21','22','23','25','26','27','29','30'
    ,'31','33','34','35','36','37','38','39','40']




In [ ]:
import scipy.io

In [ ]:
!mkdir amigos_epoched1

In [ ]:
def epoching (sub, channel, window_size, step_size, band, sample_rate):
        subject=scipy.io.loadmat('/content/drive/MyDrive/Data/'+ "Data_Preprocessed_P"+sub+"/Data_Preprocessed_P"+sub+"/Data_Preprocessed_P"+sub+ '.mat')
        signal = []
        for i in range (0,20):
            # loop over 0-39 trials
            data = subject['joined_data'][0][i]
            labels = subject['labels_selfassessment'][0][i][0]
            #print(labels)
            valence,arousal = map_label(labels)
            labels_v = np.array(valence)
            labels_a = np.array(arousal)
            # print(valence,arousal)
            start = 0;
            
            #print(data.shape[0])
            data=np.array(data)
            while start + window_size < data.shape[0]:
                meta_array = []
                meta_data = []
                for j in channel:
                    X=[]
                    for k in range(start,start+window_size):
                      X .append(data[k][j])
                    #X=data[start:start+window_size][j]
                    #print(len(X)) 
                    Y = fft(X,band,sample_rate)
                    #print(X.shape)
                    meta_data=meta_data+list(Y[0])
                    #meta_data.append(np.array(X).ravel())
                    #meta_data=np.array(meta_data).ravel()

                meta_array.append(np.array(meta_data))
                meta_array.append(labels_v)
                meta_array.append(labels_a)
                signal.append(np.array(meta_array, dtype=object))
                start = start + step_size
                # print(signal)
                
        signal = np.array(signal)
        #print(signal)
        np.save('/content/amigos_epoched1/s' + sub, signal, allow_pickle=True, fix_imports=True)


In [ ]:
band=[4, 8, 12, 16, 25, 45]
sample_rate=128
for subjects in subject_list:
    epoching (subjects, channel, window_size, step_size, band, sample_rate)
    data = []
    label_v = []
    label_a=[]
    with open('/content/amigos_epoched1/s' + subjects + '.npy', 'rb') as file:
            sub = np.load(file,allow_pickle=True)
            #print(sub)
            for i in range (0,sub.shape[0]):

              data.append(sub[i][0].ravel())
              #label.append(sub[i][1])
              label_v.append(sub[i][1].item())
              label_a.append(sub[i][2].item())
    np.save('/content/drive/MyDrive/eegEmotionDetection/amigos_combined/data_valence_arousal_binary_classif_1_'+subjects, np.array(data), allow_pickle=True, fix_imports=True)
    np.save('/content/drive/MyDrive/eegEmotionDetection/amigos_combined/label_valence_binary_classif_1_'+subjects, np.array(label_v), allow_pickle=True, fix_imports=True)
    np.save('/content/drive/MyDrive/eegEmotionDetection/amigos_combined/label_arousal_binary_classif_1_'+subjects, np.array(label_a), allow_pickle=True, fix_imports=True)
    

In [ ]:
subject_list=['01','02','03','04','05','06','07','09','10'
    ,'11','12','13','14','15','16','17','18','19','20'
    ,'21','22','23','25','26','27','29','30'
    ,'31','33','34','35','36','37','38','39','40']

In [ ]:
def fuzzy_rank(CF, top):
    R_L = np.zeros(CF.shape)
    for i in range(CF.shape[0]):
        for j in range(CF.shape[1]):
            for k in range(CF.shape[2]):
                R_L[i][j][k] = 1 - math.exp(-math.exp(-2.0*CF[i][j][k]))  #Gompertz Function
    
    K_L = 0.632*np.ones(shape = R_L.shape) #initiate all values as penalty values
    for i in range(R_L.shape[0]):
        for sample in range(R_L.shape[1]):
            for k in range(top):
                a = R_L[i][sample]
                idx = np.where(a==np.partition(a, k)[k])
                #if sample belongs to top 'k' classes, R_L =R_L, else R_L = penalty value
                K_L[i][sample][idx] = R_L[i][sample][idx]

    return K_L

def CFS_func(CF, K_L):
    H = CF.shape[0] #no. of classifiers
    for f in range(CF.shape[0]):
        for i in range(CF.shape[1]):
            idx = np.where(K_L[f][i] == 0.632)
            CF[f][i][idx] = 0
    CFS = 1 - np.sum(CF,axis=0)/H
    return CFS

def Gompertz(top = 2, *argv):
    L = 0 #Number of classifiers
    for arg in argv:
        L += 1

    num_classes = arg.shape[1]
    CF = np.zeros(shape = (L,arg.shape[0], arg.shape[1]))

    for i, arg in enumerate(argv):
        CF[:][:][i] = arg

    R_L = fuzzy_rank(CF, top) #R_L is with penalties
    
    RS = np.sum(R_L, axis=0)
    CFS = CFS_func(CF, R_L)
    FS = RS*CFS

    predictions = np.argmin(FS,axis=1)
    return predictions

In [ ]:
def predicting(ensemble_prob):
    prediction = np.zeros((ensemble_prob.shape[0],))
    for i in range(ensemble_prob.shape[0]):
        temp = ensemble_prob[i]
        t = np.where(temp == np.max(temp))[0][0]
        prediction[i] = t
    return prediction

In [ ]:
accuracies=[]
f1_arr=[]
acc_cnn=[]
acc_lstm=[]
acc_gru=[]
f1_lstm=[]
f1_cnn=[]
f1_gru=[]

In [ ]:
for i in range(20,36):
    subjects=subject_list[i]
    with open('/content/drive/MyDrive/eegEmotionDetection/amigos_combined/data_valence_arousal_binary_classif_1_'+subjects+ '.npy', 'rb') as fileTrain:
      data  = np.load(fileTrain)
    
    with open('/content/drive/MyDrive/eegEmotionDetection/amigos_combined/label_arousal_binary_classif_1_'+subjects+ '.npy', 'rb') as fileTrainL:
      label  = np.load(fileTrainL)
    x_train, x, y_train, y = train_test_split(data, label, test_size=0.4, random_state=12)
    x_val, x_test, y_val, y_test = train_test_split(x, y, test_size=0.5, random_state=12)
    unique, counts = np.unique(y_train, return_counts=True)
    label_dist = dict(zip(unique, counts))
    print(label_dist)
    unique, counts = np.unique(y_val, return_counts=True)
    label_dist = dict(zip(unique, counts))
    print(label_dist)
    unique, counts = np.unique(y_test, return_counts=True)
    label_dist = dict(zip(unique, counts))
    print(label_dist)
    scaler = StandardScaler()
    x_train= np.nan_to_num(x_train) 
    x_test= np.nan_to_num(x_test) 
    x_val= np.nan_to_num(x_val) 
    # 4 Normalization -> Standardization 
    x_train = scaler.fit_transform(normalize(x_train))
    x_val = scaler.transform(normalize(x_val))
    x_test = scaler.transform(normalize(x_test))
    x_train = x_train.reshape(x_train.shape[0],x_train.shape[1], 1)
    batch_size = 256
    num_classes = 2
    epochs = 100
    factor = 0.01
    rate = 0.2
    input_shape=(x_train.shape[1],x_train.shape[2])
    model_lstm = Sequential()

    model_lstm.add(Conv1D(64, 1, activation='relu', input_shape=input_shape))
    model_lstm.add(MaxPooling1D(pool_size=2))
    model_lstm.add(Dropout(rate))

    model_lstm.add(Conv1D(128, 1, activation='relu'))
    model_lstm.add(MaxPooling1D(pool_size=2))
    model_lstm.add(Dropout(rate))

    model_lstm.add(Conv1D(256, 1, activation='relu'))
    model_lstm.add(MaxPooling1D(pool_size=2))
    model_lstm.add(Dropout(rate))

    model_lstm.add(LSTM(units = 256, return_sequences=True))  
    model_lstm.add(Dropout(rate))

    model_lstm.add(LSTM(units = 128, return_sequences=True))
    model_lstm.add(Dropout(rate))

    model_lstm.add(LSTM(units = 64))
    model_lstm.add(Dropout(rate))

    # model_lstm.add(Flatten())

    model_lstm.add(Dense(units = 32, activation='relu'))
    model_lstm.add(Dropout(rate))

    model_lstm.add(Dense(units = 1))
    model_lstm.add(Activation('sigmoid'))

    model_lstm.summary()
    model_lstm.compile(
    optimizer =Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07),
    loss = 'binary_crossentropy',
    metrics=["accuracy"]
    )
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3,
                              patience=5, min_lr=2.5e-5, mode='min')
    
    history_lstm = model_lstm.fit(
    x_train, 
    y_train,
    epochs=50,
    batch_size=batch_size,
    verbose=1,
    validation_data=(x_val, y_val),callbacks=[reduce_lr]
    )
    y_pred_prob = model_lstm.predict(x_test, verbose=1)
    y_pred = (y_pred_prob>0.5)*1 #or greater than equal to?
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    acc_lstm.append(acc)
    f1_lstm.append(f1)

    model_gru = Sequential()

    model_gru.add(Conv1D(64, 3, activation='relu', input_shape=input_shape))
    model_gru.add(MaxPooling1D(pool_size=2))
    model_gru.add(Dropout(rate))

    model_gru.add(Conv1D(128, 3, activation='relu'))
    model_gru.add(MaxPooling1D(pool_size=2))
    model_gru.add(Dropout(rate))

    model_gru.add(Conv1D(256, 3, activation='relu'))
    model_gru.add(MaxPooling1D(pool_size=2))
    model_gru.add(Dropout(rate))

    model_gru.add(GRU(units = 256, return_sequences=True))  
    model_gru.add(Dropout(rate))

    model_gru.add(GRU(units = 128, return_sequences=True))
    model_gru.add(Dropout(rate))

    model_gru.add(GRU(units = 64))
    model_gru.add(Dropout(rate))

    # model_gru.add(Flatten())

    model_gru.add(Dense(units = 32, activation='relu'))
    model_gru.add(Dropout(rate))

    model_gru.add(Dense(units = 1))
    model_gru.add(Activation('sigmoid'))

    model_gru.summary()
    model_gru.compile(
    optimizer =Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07),
    loss = 'binary_crossentropy',
    metrics=["accuracy"]
    )
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3,
                              patience=5, min_lr=2.5e-5, mode='min')
    history_gru = model_gru.fit(
    x_train, 
    y_train,
    epochs=50,
    batch_size=batch_size,
    verbose=1,
    validation_data=(x_val, y_val), #,
    callbacks=[reduce_lr]
    # callbacks=[
    #     keras.callbacks.EarlyStopping(
    #         monitor='val_loss',
    #         patience=20,
    #         restore_best_weights=True
    #     )
    # ]
    )
    y_pred_prob = model_gru.predict(x_test, verbose=1)
    y_pred = (y_pred_prob>0.5)*1 #or greater than equal to?
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    acc_gru.append(acc)
    f1_gru.append(f1)
    model_cnn = Sequential()

    model_cnn.add(Conv1D(64, 3, activation='relu', input_shape=input_shape))
    model_cnn.add(MaxPooling1D(pool_size=2))
    model_cnn.add(Dropout(rate))

    model_cnn.add(Conv1D(128, 3, activation='relu', input_shape=input_shape))
    model_cnn.add(MaxPooling1D(pool_size=2))
    model_cnn.add(Dropout(rate))

    model_cnn.add(Conv1D(256, 3, activation='relu', input_shape=input_shape))
    model_cnn.add(MaxPooling1D(pool_size=2))
    model_cnn.add(Dropout(rate))

    model_cnn.add(Conv1D(512, 3, activation='relu', input_shape=input_shape))
    model_cnn.add(MaxPooling1D(pool_size=2))
    model_cnn.add(Dropout(rate))

    model_cnn.add(Flatten())

    model_cnn.add(Dense(units = 512, activation='relu'))
    model_cnn.add(Dropout(rate))

    model_cnn.add(Dense(units = 256, activation='relu'))
    model_cnn.add(Dropout(rate))

    model_cnn.add(Dense(units = 128, activation='relu'))
    model_cnn.add(Dropout(rate))

    model_cnn.add(Dense(units = 1))
    model_cnn.add(Activation('sigmoid'))

    model_cnn.summary()
    model_cnn.compile(
    optimizer =Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07),
    loss = 'binary_crossentropy',
    metrics=["accuracy"]
    )
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3,
                              patience=5, min_lr=2.5e-5, mode='min')
    history_cnn = model_cnn.fit(
    x_train, 
    y_train,
    epochs=50,
    batch_size=batch_size,
    verbose=1,
    validation_data=(x_val, y_val), #,
    callbacks=[reduce_lr]
    # callbacks=[
    #     keras.callbacks.EarlyStopping(
    #         monitor='val_loss',
    #         patience=20,
    #         restore_best_weights=True
    #     )
    # ]
    )
    y_pred_prob = model_cnn.predict(x_test, verbose=1)
    y_pred = (y_pred_prob>0.5)*1 #or greater than equal to?
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    acc_cnn.append(acc)
    f1_cnn.append(f1)
    y_pred_prob_lstm = model_lstm.predict(x_test, verbose=1)
    y_pred_lstm = (y_pred_prob_lstm>0.5)*1
    y_pred_prob_gru = model_gru.predict(x_test, verbose=1)
    y_pred_gru = (y_pred_prob_gru>0.5)*1
    y_pred_prob_cnn = model_cnn.predict(x_test, verbose=1)
    y_pred_cnn = (y_pred_prob_cnn>0.5)*1
    p1 = y_pred_prob_lstm
    p2 = y_pred_prob_gru
    p3 = y_pred_prob_cnn
    p1 = np.concatenate([1-p1,p1],axis=1)
    p2 = np.concatenate([1-p2,p2],axis=1)
    p3 = np.concatenate([1-p3,p3],axis=1)
    #Calculate Gompertz Function Ensemble
    top = 2 #top 'k' classes
    predictions = Gompertz(top, p1, p2, p3)
    y_pred = predictions
    acc = accuracy_score(y_test, y_pred)
    accuracies.append(acc)
    print('Acc :',acc)
    f1 = f1_score(y_test, y_pred)
    f1_arr.append(f1)
    print('F1 :',f1)
    precision = precision_score(y_test, y_pred)
    print('Precision :',precision)
    recall = recall_score(y_test, y_pred)
    print('Recall :',recall)
    # classLabels = np.unique(np.argmax(y_train,axis=1))
    f1_scores = f1_score(y_test, y_pred)
    # f1_scores_with_labels = {label:score for label,score in zip(classLabels, f1_scores)}
    # print('Classwise F1 :\n',f1_scores_with_labels)
    # classLabels = ['0','1']
    cnfMatrix = confusion_matrix(y_test, y_pred) #change
    print('Confusion Matrix :\n',cnfMatrix)
    



Streaming output truncated to the last 5000 lines.
 dropout_689 (Dropout)       (None, 7, 256)            0         
                                                                 
 gru_97 (GRU)                (None, 7, 128)            148224    
                                                                 
 dropout_690 (Dropout)       (None, 7, 128)            0         
                                                                 
 gru_98 (GRU)                (None, 64)                37248     
                                                                 
 dropout_691 (Dropout)       (None, 64)                0         
                                                                 
 dense_260 (Dense)           (None, 32)                2080      
                                                                 
 dropout_692 (Dropout)       (None, 32)                0         
                                                                 
 dense_261 (Dense)       

In [ ]:
sum=0

In [ ]:
for acc in accuracies:
  #print(acc)
  sum+=acc
print(sum)

35.519838872104735


In [ ]:
print(sum/36)

0.9866621908917982


In [ ]:
sum1=0
for f in f1_arr:
  sum1+=f
print(sum1)
print(sum1/36)

35.47737643061835
0.9854826786282875


In [ ]:
sum=0
for acc in acc_cnn:
  sum+=acc
print(sum/36)

0.9856719257021371


In [ ]:
sum=0
for f in f1_cnn:
  sum+=f
print(sum/36)

0.984448062679358


In [ ]:
sum=0
for acc in acc_lstm:
  sum+=acc
print(sum/36)

0.9522378874342622


In [ ]:
sum=0
for acc in acc_gru:
  sum+=acc
print(sum/36)

0.9905001678415578


In [ ]:
sum=0
for f in f1_cnn:
  sum+=f
print(sum/36)

0.984448062679358


In [ ]:
sum=0
for f in f1_lstm:
  sum+=f
print(sum/36)

0.9453100440689677


In [ ]:
sum=0
for f in f1_gru:
  sum+=f
print(sum/36)

0.990316477637092
